In [1]:
import os

In [2]:
def create_folder(directory:str) -> None:
	if not os.path.exists(directory):
		os.makedirs(directory)

In [3]:
root = "./../"

create_folder(f"{root}Queries")

# Queries time information

In [4]:
with open(f"{root}Queries/query_timestamps.q", "w", encoding="UTF-8") as f_w:
	f_w.write("""SELECT DISTINCT ?h ?r ?v ?point
WHERE {
        FILTER(strstarts(str(?r), "http://www.wikidata.org/prop/P"))
        FILTER(strstarts(str(?statement), "http://www.wikidata.org/entity/statement/"))
        ?h ?r ?statement.

        FILTER(strstarts(str(?rs), "http://www.wikidata.org/prop/statement/P"))
        ?statement ?rs ?v.

        FILTER(datatype(?point) = <http://www.w3.org/2001/XMLSchema#dateTime>)
        ?statement <http://www.wikidata.org/prop/qualifier/P585> ?point
}""")

In [5]:
with open(f"{root}Queries/query_intervals.q", "w", encoding="UTF-8") as f_w:
	f_w.write("""SELECT DISTINCT ?h ?r ?v ?start ?end
WHERE {
        FILTER(strstarts(str(?statement), "http://www.wikidata.org/entity/statement/"))
        FILTER(strstarts(str(?rs), "http://www.wikidata.org/prop/statement/P"))
        FILTER((strlen(str(?start))>0) || (strlen(str(?end))>0))
        ?h ?r ?statement.
        ?statement ?rs ?v.
        OPTIONAL{
                ?statement <http://www.wikidata.org/prop/qualifier/P580> ?start.
                FILTER(datatype(?start) = <http://www.w3.org/2001/XMLSchema#dateTime>)
        }
        OPTIONAL{
                ?statement <http://www.wikidata.org/prop/qualifier/P582> ?end.
                FILTER(datatype(?end) = <http://www.w3.org/2001/XMLSchema#dateTime>)
        }
}
""")

# Class per entities retrieved

In [6]:
entities = set()
for file in ["result_intervals.quintuplet", "result_timestamps.quad"]:
	with open(f"{root}Data/Raw/{file}", "r", encoding="UTF-8") as f_r:
		line = f_r.readline()
		while line != "":
			h, _, v = line.split("\t")[:3]

			entities.add(h[:-1].split("/")[-1])

			iri_prefix = "<http://www.wikidata.org/entity/Q"
			if v[:len(iri_prefix)] == iri_prefix:
				entities.add(v[:-1].split("/")[-1])
				
			line = f_r.readline()

with open(f"{root}Queries/query_classes_per_entities.q", "w", encoding="UTF-8") as f_w:
	f_w.write("""
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
		   
SELECT ?entity ?class
WHERE {
	VALUES ?entity { wd:""")
	f_w.write(" wd:".join(list(entities)[:100]))
	f_w.write("""}
	?entity wdt:P31 ?class.
}
""")

# Hierarchy of classes 

In [8]:
with open(f"{root}Queries/query_hierarchy_classes.q", "w", encoding="UTF-8") as f_w:

	f_w.write("""
	SELECT ?sub ?up
	WHERE {
			?sub <http://www.wikidata.org/prop/direct/P279> ?up.
	}
	""")